# JPMorgan Chase & Co. Quantitative Research Job Simulation
## Task 2

In [33]:
from datetime import date
from typing import List, Tuple
from math import ceil

We model the price of the contract as follows:
- Let the volume of gas be denoted by $V$ and the maximum volume that can be stored is $V_{max}$.
- We assume that the volume of gas injected/withdrawn is the same.
- As the transport costs and payments to the owner at each injection/withdrawal occurs during each event, we can lump them together as $C$.
- The volume injected/withdrawn during that event would simply be equal to the rate of volume injection/withdrawal, multiplied by the time duration for the injection/withdrawal which is assumed to be constant at one day long and the rate of volume is given per day.
- Let $I$ denote the set of injection events where $I \in \{1, 2, ..., N_I\}$ where $N_I$ is the total number of injection events.
    - $t_i$ = date (time) of the $i$-th injection.
    - $\Delta V_i$ = rate of volume injection.
    - $P(t_i)$ = market price per MMBtu at the $i$-th injection.
    - $C(\Delta V_i)$ = transport and owner costs for that volume at $i$-th injection.
- Let $J$ denote the set of withdrawal events where $J \in \{1, 2, ..., N_J\}$ where $N_J$ is the total number of withdrawal events.
    - $t_j$ = date (time) of the $j$-th withdrawal.
    - $\Delta V_j$ = rate of volume withdrawal.
    - $P(t_j)$ = market price per MMBtu at the $j$-th withdrawal.
    - $C(\Delta V_j)$ = transport and owner costs for that volume at $j$-th withdrawal.
- Let $K$ denote the set of billing events where $K \in \{1, 2, ..., N_K\}$ where $N_K$ is the total number of monthly billing periods.
    - $S_k$ = storage costs for the $k$-th month for the contract.

We assume that the $\Delta V_i = \Delta V_j$ since injection and withdrawal volumes are the same, hence the change in volume each day can be denoted by $\Delta V$.

Therefore, we have the following:
$$
    \text{contact cost} = \sum_{j \in J}{[P(t_j) \cdot \Delta V - C(\Delta V)]} - \sum_{i \in I}{[P(t_i) \cdot \Delta V + C(\Delta V)]} - \sum_{k \in K}S_k
$$

In [46]:
def commodity_storage_contract(
        injections: List[Tuple],
        withdrawals: List[Tuple],
        del_V: float,
        V_max: float,
        S: float
)-> float:

    # Define the constant cost parameters per transcation
    OWNER_RATE = 0.005          # USD per volume paid to owner
    TRANSPORT_COST = 50000      # USD for injection/withdrawal

    # Volume of gas injected/withdrawn in a transaction (assume rate is daily)
    del_V = volume_rate

    # We compute the total cost for transporting and owner payments
    C = TRANSPORT_COST + OWNER_RATE * del_V

    # Combine the injection and withdrawal contracts
    contracts = injections + withdrawals

    # Sort by dates
    contracts = sorted(contracts, key=lambda x: x[0])

    # Get the start and end dates
    start_date = contracts[0][0]
    end_date = contracts[-1][0]

    # Find the duration of the contract in days and months
    delta_T = end_date - start_date
    T_days = delta_T.days
    T_months = ceil(T_days / 30)

    # Initialise total values
    J_net = 0.0                 # Net withdraws
    I_net = 0.0                 # Net injections
    V = 0.0                     # Volume of gas stored

    # Compute injections
    for contract in contracts:
        date, price, contract_type = contract

        # Compute injections
        if contract_type == 'inject':
            if V <= V_max + del_V:
                I_net += price * del_V + C
                V += del_V
            else:
                 # If volume exceeds maximum capacity
                print(f'Skipping contract with date {date} as volume of {V} exceeds the maximum capacity of {V_max}.')

        # Compute withdrawals
        elif contract_type == 'withdraw':
            if V >= del_V:
                J_net += price * del_V - C
                V -= del_V
            else:
                # If volume is negative
                print(f'Skipping contract with data {date} as volume of {V} is too low.')

        print(f'We {contract_type} {del_V:.2f} MMBtu of natural gas at ${price:.2f} on {date} with {V:.2f} MMBtu gas left in storage.\n')
    
    # Consider all costs
    contract_value = J_net - I_net - S * T_months

    return contract_value

# Validate the contract with some test data
injections = [
    (date(2022, 1, 1), 20, 'inject'),
    (date(2022, 2, 1), 21, 'inject'),
    (date(2022, 2, 21), 18.5, 'inject'),
    (date(2022, 4, 1), 22, 'inject')
]
withdrawals = [
    (date(2022, 1, 27), 23, 'withdraw'),
    (date(2022, 2, 15), 19, 'withdraw'),
    (date(2022, 3, 20), 21, 'withdraw'),
    (date(2022, 6, 1), 25, 'withdraw')
]
volume_rate = 100000
storage_cost_rate = 10000
maximum_storage_volume = 500000
result = commodity_storage_contract(injections, withdrawals, volume_rate, maximum_storage_volume, storage_cost_rate)
print(f'The value of the contract is ${result:.2f}.')

We inject 100000.00 MMBtu of natural gas at $20.00 on 2022-01-01 with 100000.00 MMBtu gas left in storage.

We withdraw 100000.00 MMBtu of natural gas at $23.00 on 2022-01-27 with 0.00 MMBtu gas left in storage.

We inject 100000.00 MMBtu of natural gas at $21.00 on 2022-02-01 with 100000.00 MMBtu gas left in storage.

We withdraw 100000.00 MMBtu of natural gas at $19.00 on 2022-02-15 with 0.00 MMBtu gas left in storage.

We inject 100000.00 MMBtu of natural gas at $18.50 on 2022-02-21 with 100000.00 MMBtu gas left in storage.

We withdraw 100000.00 MMBtu of natural gas at $21.00 on 2022-03-20 with 0.00 MMBtu gas left in storage.

We inject 100000.00 MMBtu of natural gas at $22.00 on 2022-04-01 with 100000.00 MMBtu gas left in storage.

We withdraw 100000.00 MMBtu of natural gas at $25.00 on 2022-06-01 with 0.00 MMBtu gas left in storage.

The value of the contract is $186000.00.
